## Anomaly Detection in Multi Dimensional Signal

In [ ]:
# Uncomment to install Analytics package
# !pip install git+https://github.com/cognetive/VPC-Flow-Logs-Analytics.git

In [ ]:
# Run this cell after loading flowlogs_df
from data_preparation import data_format

flowlogs_df = data_format(flowlogs_df)

### Prepare Data

In [ ]:
# metrics are the columns in flowlogs_df where anomalies should be detected.
# sample_rate is the rate these columns will be resampled and aggregated.
metrics = ['ABBytes', 'BABytes', 'ABPackets', 'BAPackets'] 
sample_rate = '1Min'

In [ ]:
from data_preparation import time_series

data_df = time_series(flowlogs_df, metrics, sample_rate)

In [ ]:
# train_size is the relative size of the training set.
train_size = 0.8 

In [ ]:
from data_preparation import standardize

print("Preparing data to learning task...")
split_index = int(train_size*len(data_df.index))
train_set = standardize(data_df[:split_index])
test_set = standardize(data_df[split_index:])
print("Done.")

### Decode-Encode

In [ ]:
epochs = 1   # Duration of training.

In [ ]:
from machine_learning import FC_autoencoder

decoded, encoded = FC_autoencoder(train_set, test_set, max_epochs=epochs)

### Anomaly Times

In [ ]:
import numpy as np
from anomaly_processing import find_anomalies

scores = [np.mean(np.abs(test_set.iloc[i] - encoded.iloc[i])) for i in range(len(test_set.index))]
anomaly_indices = find_anomalies(scores)
print("Anomalies occured at:")
for ind in anomaly_indices:
    print(test_set.index[ind])